# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 655475652849067168
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 235929600
locality {
  bus_id: 1
}
incarnation: 10853659512003762676
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = len(max(x, key=len))

    return pad_sequences(x, maxlen=length, padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)
print(preproc_english_sentences.shape)
print(preproc_french_sentences.shape)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344
(137861, 15)
(137861, 21, 1)


In [10]:
preproc_english_sentences.shape

(137861, 15)

In [11]:
preproc_french_sentences.shape

(137861, 21, 1)

In [13]:
# another_arr = arr.reshape(-1, arr.shape[-1])


In [14]:
preproc_french_sentences.shape

(137861, 21, 1)

In [15]:
preproc_french_sentences = preproc_french_sentences.reshape(preproc_french_sentences.shape[:1])
preproc_french_sentences.shape

ValueError: cannot reshape array of size 2895081 into shape (137861,)

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [16]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [48]:
tmp_x.shape

(137861, 15)

In [50]:
input_seq = Input(tmp_x.shape[1:])
input_seq.shape

TensorShape([Dimension(None), Dimension(15)])

In [52]:
input_seq = Input(shape=(max_french_sequence_length, 1))
input_seq.shape

TensorShape([Dimension(None), Dimension(21), Dimension(1)])

In [13]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 74us/step - loss: 3.4704 - acc: 0.4149 - val_loss: nan - val_acc: 0.4586
Epoch 2/10
110288/110288 [==============================] - 6s 57us/step - loss: 2.4391 - acc: 0.4711 - val_loss: nan - val_acc: 0.4837
Epoch 3/10
110288/110288 [==============================] - 6s 57us/step - loss: 2.1875 - acc: 0.5045 - val_loss: nan - val_acc: 0.5288
Epoch 4/10
110288/110288 [==============================] - 6s 57us/step - loss: 1.9392 - acc: 0.5523 - val_loss: nan - val_acc: 0.5686
Epoch 5/10
110288/110288 [==============================] - 6s 57us/step - loss: 1.7690 - acc: 0.5748 - val_loss: nan - val_acc: 0.5789
Epoch 6/10
110288/110288 [==============================] - 6s 56us/step - loss: 1.6723 - acc: 0.5796 - val_loss: nan - val_acc: 0.5820
Epoch 7/10
110288/110288 [==============================] - 6s 56us/step - loss: 1.6110 - acc: 0.5849 - val_loss: nan - val_acc: 0.59

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    embed_layer = Embedding(english_vocab_size, 64, input_length=output_sequence_length)(input_seq)
    rnn = GRU(64, return_sequences=True)(embed_layer)

    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 69us/step - loss: 3.8080 - acc: 0.4072 - val_loss: nan - val_acc: 0.4095
Epoch 2/10
110288/110288 [==============================] - 7s 65us/step - loss: 2.5855 - acc: 0.4677 - val_loss: nan - val_acc: 0.5160
Epoch 3/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.8610 - acc: 0.5774 - val_loss: nan - val_acc: 0.6265
Epoch 4/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.3768 - acc: 0.6678 - val_loss: nan - val_acc: 0.7120
Epoch 5/10
110288/110288 [==============================] - 7s 65us/step - loss: 1.1166 - acc: 0.7350 - val_loss: nan - val_acc: 0.7545
Epoch 6/10
110288/110288 [==============================] - 7s 65us/step - loss: 0.9509 - acc: 0.7675 - val_loss: nan - val_acc: 0.7787
Epoch 7/10
110288/110288 [==============================] - 7s 64us/step - loss: 0.8303 - acc: 0.7872 - val_loss: nan - val_acc: 0.79

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [13]:
preproc_french_sentences.shape

(137861, 21, 1)

In [14]:
preproc_english_sentences.shape

(137861, 15)

In [32]:
max_french_sequence_length

21

In [17]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    print(input_seq.shape)
    bdrnn = Bidirectional(GRU(64, return_sequences=True))(input_seq)
    print(bdrnn.shape)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(bdrnn)
    print(logits.shape)    
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_bd_model(bd_model)

# TODO: Train and Print prediction(s)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bdrnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bdrnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=1, validation_split=0.2)

print(logits_to_text(bdrnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(?, 21, 1)
(?, ?, 128)
(?, 21, 344)
(?, 21, 1)
(?, ?, 128)
(?, 21, 344)
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
110288/110288 [==============================] - 12s 108us/step - loss: 3.2254 - acc: 0.4621 - val_loss: nan - val_acc: 0.5189
new l' est est est est est mais est est en en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [39]:
max_english_sequence_length

15

In [14]:
from keras.layers import LSTM

In [22]:
preproc_french_sentences.shape

(137861, 21, 1)

In [43]:
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2], 1))
print(tmp_x.shape)

tmp_y = pad(preproc_french_sentences, max_french_sequence_length)
# tmp_y = tmp_y.reshape((-1, preproc_french_sentences.shape[-2], 1))
print(tmp_y.shape)

(137861, 15)
(137861, 21)


In [33]:
tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
tmp_x.shape

(137861, 15)

In [41]:
INPUT_LENGTH = tmp_x.shape[1]
OUTPUT_LENGTH = tmp_y.shape[1]

encoder_inputs = Input(shape=(INPUT_LENGTH,))
print(encoder_inputs.shape)
en_emb = Embedding(english_vocab_size, 64, input_length=INPUT_LENGTH)(encoder_inputs)
print(en_emb.shape)
encoder_outputs, state_h, state_c = LSTM(64, return_sequences=True, return_state=True)(en_emb)
print(encoder_outputs.shape)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(OUTPUT_LENGTH,))
print(decoder_inputs.shape)
de_emb = Embedding(french_vocab_size, 64, input_length=OUTPUT_LENGTH)(decoder_inputs)
print(de_emb.shape)
decoder_outputs, _, _ = LSTM(64, return_sequences=True, return_state=True)(de_emb, initial_state=encoder_states)
logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder_outputs)
print(logits.shape)


(?, 15)
(?, 15, 64)
(?, ?, 64)
(?, 21)
(?, 21, 64)
(?, 21, 344)


In [46]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    learning_rate = 1e-3
    INPUT_LENGTH = output_sequence_length
    OUTPUT_LENGTH = output_sequence_length
    
    encoder_inputs = Input(shape=(INPUT_LENGTH,))
    print(encoder_inputs.shape)
    en_emb = Embedding(english_vocab_size, 64, input_length=INPUT_LENGTH)(encoder_inputs)
    print(en_emb.shape)
    encoder_outputs, state_h, state_c = LSTM(64, return_sequences=True, return_state=True)(en_emb)
    print(encoder_outputs.shape)
    encoder_states = [state_h, state_c]
    
    decoder_inputs = Input(shape=(OUTPUT_LENGTH,))
    print(decoder_inputs.shape)
    de_emb = Embedding(french_vocab_size, 64, input_length=OUTPUT_LENGTH)(decoder_inputs)
    print(de_emb.shape)
    decoder_outputs, _, _ = LSTM(64, return_sequences=True, return_state=True)(de_emb, initial_state=encoder_states)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder_outputs)
    
    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=logits)
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)

# OPTIONAL: Train and Print prediction(s)

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2], 1))
# print(tmp_x.shape)

tmp_y = pad(preproc_french_sentences, max_french_sequence_length)
# tmp_y = tmp_y.reshape((-1, preproc_french_sentences.shape[-2], 1))
# print(tmp_y.shape)

# TODO: Train the neural network
encdec_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, tmp_y, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(?, 21)
(?, 21, 64)
(?, ?, 64)
(?, 21)
(?, 21, 64)


AssertionError: Wrong input shape. Found input shape [(None, 21), (None, 21)] using parameter input_shape=(137861, 15, 1)

In [66]:
preproc_english_sentences.shape

(137861, 15)

In [67]:
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
print(tmp_x.shape)

# tmp_y = pad(preproc_french_sentences, max_french_sequence_length)
# tmp_y = tmp_y.reshape((-1, preproc_french_sentences.shape[-2], 1))
# print(tmp_y.shape)
print(preproc_french_sentences.shape)

(137861, 21)
(137861, 21, 1)
(137861, 21, 1)


In [70]:
# don't use the embedding layer
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    learning_rate = 1e-3
    
    inputs = Input(input_shape[1:])
    encoder = LSTM(64)(inputs)
    decoder = RepeatVector(output_sequence_length)(encoder)
    decoder = LSTM(64, return_sequences=True)(decoder)
    decoded = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)

    model = Model(inputs=inputs, outputs=decoded)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model

tests.test_encdec_model(encdec_model)

# OPTIONAL: Train and Print prediction(s)

encdec_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

encdec_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
# print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
109568/110288 [============================>.] - ETA: 1s - loss: 3.4632 - acc: 0.4185

InvalidArgumentError: Received a label value of 344 which is outside the valid range of [0, 344).  Label values: 7 88 1 64 16 13 14 5 7 83 1 40 13 14 0 0 0 0 0 0 0 11 30 1 67 15 25 22 47 6 3 1 58 69 2 56 0 0 0 0 0 0 62 1 92 2 41 5 3 1 9 21 26 11 24 0 0 0 0 0 0 0 0 7 33 1 9 7 102 2 51 5 3 1 18 2 45 0 0 0 0 0 0 0 29 1 65 37 11 44 6 3 1 9 21 2 52 0 0 0 0 0 0 0 0 10 38 1 20 16 17 13 5 10 82 1 20 13 17 0 0 0 0 0 0 0 36 1 12 67 37 11 46 5 3 1 18 2 44 0 0 0 0 0 0 0 0 27 145 7 97 134 154 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 113 115 184 110 176 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 94 4 28 4 73 6 4 78 0 0 0 0 0 0 0 0 0 0 0 0 11 30 1 19 15 25 79 56 5 3 1 68 2 41 0 0 0 0 0 0 0 3 55 4 72 4 28 76 6 4 28 0 0 0 0 0 0 0 0 0 0 0 3 55 4 78 4 74 6 4 73 0 0 0 0 0 0 0 0 0 0 0 0 3 55 4 28 4 70 6 4 72 0 0 0 0 0 0 0 0 0 0 0 0 11 30 1 12 57 15 25 22 45 6 3 1 12 18 2 47 0 0 0 0 0 4 32 31 1 8 68 15 25 22 47 5 3 1 8 21 2 41 0 0 0 0 10 82 1 61 16 17 5 7 86 1 40 93 0 0 0 0 0 0 0 0 0 10 85 1 20 16 17 13 5 7 83 1 61 13 17 0 0 0

In [68]:
30/2000

0.015

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [23]:
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x.shape

(137861, 21)

In [29]:
french_vocab_size

344

In [34]:
tmp_x.shape[1]

21

In [38]:
max_french_sequence_length

21

In [37]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate=1e-3

    input_seq = Input(input_shape[1:])
#     print(input_seq.shape)
    emb = Embedding(english_vocab_size, 64, input_length=output_sequence_length)(input_seq)
#     print(emb.shape)

    bdrnn = Bidirectional(GRU(64, return_sequences=True))(emb)
#     print(bdrnn.shape)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(bdrnn)
#     print(logits.shape)

    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_model_final(model_final)

print('Final Model Loaded')
# TODO: Train the final model

# Pad the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train
emb_bdrnn = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

emb_bdrnn.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=1, validation_split=0.2)

(?, 15)
(?, 15, 64)
(?, ?, 128)
(?, 21, 344)
Final Model Loaded
(?, 21)
(?, 21, 64)
(?, ?, 128)
(?, 21, 344)
Train on 110288 samples, validate on 27573 samples
Epoch 1/1
  5120/110288 [>.............................] - ETA: 4:42 - loss: 5.8096 - acc: 0.2547 

InvalidArgumentError: indices[550,0] = 199 is not in [0, 199)
	 [[Node: embedding_14/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_14/embeddings/read, embedding_14/Cast)]]

Caused by op 'embedding_14/Gather', defined at:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  

## Prediction (IMPLEMENTATION)

In [71]:
preproc_english_sentences.shape

(137861, 15)

In [72]:
preproc_french_sentences.shape

(137861, 21, 1)

In [73]:
preproc_french_sentences.shape[-1]

1

In [75]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate=1e-3

    input_seq = Input(input_shape[1:])
#     print(input_seq.shape)
    emb = Embedding(english_vocab_size, 64, input_length=output_sequence_length)(input_seq)
#     print(emb.shape)

    bdrnn = Bidirectional(GRU(64, return_sequences=True))(emb)
#     print(bdrnn.shape)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(bdrnn)
#     print(logits.shape)

    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


In [76]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    
    max_english_sequence_length = x.shape[1]
    max_french_sequence_length = y.shape[1]
    english_vocab_size = len(x_tk.word_index)
    french_vocab_size = len(y_tk.word_index)

    # Pad the input
    x = pad(x, max_french_sequence_length)

    # Train
    emb_bdrnn = model_final(
        x.shape,
        max_french_sequence_length,
        english_vocab_size,
        french_vocab_size)

    emb_bdrnn.fit(x, y, batch_size=1024, epochs=1, validation_split=0.2)
    

    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/1
  4096/110288 [>.............................] - ETA: 4:53 - loss: 5.8245 - acc: 0.1196

InvalidArgumentError: indices[628,0] = 199 is not in [0, 199)
	 [[Node: embedding_31/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_31/embeddings/read, embedding_31/Cast)]]

Caused by op 'embedding_31/Gather', defined at:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [2]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 305996 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?